<a href="https://colab.research.google.com/github/SURESHBEEKHANI/Advanced-LLM-Fine-Tuning/blob/main/Deep-seek-R1-MedicalSFT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-Tuning DeepSeek-R1-Distill-Llama-8B

## Objective:
Adapt `DeepSeek-R1-Distill-Llama-8B` for medical chain-of-thought reasoning.

## Key Components:
- **Model:** `unsloth/DeepSeek-R1-Distill-Llama-8B`

> Add blockquote


- **Dataset:** 500 samples from `medical-o1-reasoning-SFT`
- **Tools:**
  - `Unsloth` (2x faster training)
  - 4-bit quantization
  - LoRA adapters
- **Result:** 44-minute training resulting in concise medical reasoning with structured `<think>` outputs.

## Performance Improvement:

| **Metric**         | **Before Fine-Tuning** | **After Fine-Tuning** |
|--------------------|------------------------|-----------------------|
| **Response Length** | 450 words              | 150 words             |
| **Reasoning Style** | Verbose                | Focused               |
| **Answer Format**   | Bulleted               | Paragraph             |


### step-by-step  fine-tune DeepSeek-R1-Distill-Llama-8B on medical data

## 1. Environment Setup

In [ ]:
!pip install unsloth
!pip install --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

## 2. Authentication

In [ ]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("HUGGINGFACE_TOKEN")
login(hf_token)

## 3. Model Initialization

In [ ]:
from unsloth import FastLanguageModel

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/DeepSeek-R1-Distill-Llama-8B",
    max_seq_length=2048,
    load_in_4bit=True,
    token=hf_token
)

## 4. Dataset Preparation

In [ ]:
from datasets import load_dataset

dataset = load_dataset(
    "FreedomIntelligence/medical-o1-reasoning-SFT",
    "en",
    split="train[0:500]",
    trust_remote_code=True
)

## 5. Prompt Formatting

In [ ]:
def formatting_prompts_func(examples):
    texts = []
    for q, cot, ans in zip(examples["Question"], examples["Complex_CoT"], examples["Response"]):
        text = f"""Below is an instruction... [truncated prompt template]""" + tokenizer.eos_token
        texts.append(text)
    return {"text": texts}

dataset = dataset.map(formatting_prompts_func, batched=True)

## 6. LoRA Configuration



In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16
)

## 7. Training Setup

In [ ]:
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    dataset_text_field="text",
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        learning_rate=2e-4,
        max_steps=60,
        fp16=True,
        output_dir="outputs"
    )
)

### 8. Start Training

In [ ]:
trainer.train()

## 9. Save & Deploy

In [ ]:
# Save locally
model.save_pretrained_merged("DeepSeek-R1-Medical-COT", tokenizer, save_method="merged_16bit")

# Push to Hub
model.push_to_hub_merged("username/DeepSeek-R1-Medical-COT", tokenizer, save_method="merged_16bit")